In [13]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Embedding, Flatten, Input, concatenate
import pickle
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import keras.backend as K
import numpy as np
import pandas as pd
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)+1e-10, axis=-1))
earlyStopping=EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')

ckpt_path = './log/weights-{val_loss:.4f}.hdf5'
ckpt = ModelCheckpoint(ckpt_path,
                       monitor='val_loss', 
                       verbose=1, 
                       save_best_only=True, 
                       mode='min')
def zuhe(line):
    line['hour'] = line['hour'].astype(str)
    line['weekday'] = line['weekday'].astype(str)
    line['s_ij'] = line['s_ij'].astype(str)
    line['e_ij'] = line['e_ij'].astype(str)

#     line['is_peek'] = line['is_peek'].astype(str)
    line['is_crowd'] = line['is_crowd'].astype(str)
                                   
    line['hour_weekday'] = line['hour'] + line['weekday']
    line['hour_crowd'] = line['hour'] + line['is_crowd']
    line['hour_s'] = line['hour'] + line['s_ij']
    line['hour_e'] = line['hour'] + line['e_ij']

    line['hour'] = line['hour'].astype(int)
#     line['is_peek'] = line['is_peek'].astype(int)
    line['is_crowd'] = line['is_crowd'].astype(int)
    line['weekday'] = line['weekday'].astype(int)
    line['s_ij'] = line['s_ij'].astype(int)
    line['e_ij'] = line['e_ij'].astype(int)
    
    line['hour_weekday'] = line['hour_weekday'].astype(int)
    line['hour_crowd'] = line['hour_crowd'].astype(int)
    line['hour_s'] = line['hour_s'].astype(int)
    line['hour_e'] = line['hour_e'].astype(int)
    return line

with open('./data/train-id4-crowd-grid3.txt', 'rb') as data_file:
    train = pickle.load(data_file)
    #训练验证都划分
    #900 29 26 #800 27.9 25.9 #700 27.5 25.3 #27.4 24.8 #26.4 24.08 #23 21 #(90)250 21 19.5 #(80)17. 16.5
    #训练划分 验证不划分
    #250
train = train[(train['Diff_Time']<600)]
train = zuhe(train)
with open('./data/test-id4-crowd-grid3.txt', 'rb') as data_file:
    test = pickle.load(data_file)
test = zuhe(test)

from sklearn import preprocessing
x11 = train['ID'].as_matrix().reshape([-1,1])
x12 = train['e_ij'].as_matrix().reshape([-1,1])
x13 = train['s_ij'].as_matrix().reshape([-1,1])

N1 = x11.shape[0]

x21 = test['ID'].as_matrix().reshape([-1,1])
x22 = test['e_ij'].as_matrix().reshape([-1,1])
x23 = test['s_ij'].as_matrix().reshape([-1,1])

N2 = x21.shape[0]

x1 = np.concatenate((x11,x21))
x1 = preprocessing.LabelEncoder().fit_transform(x1) #13963746x21567
print(x1.shape)#(14454126, 21567)
train['new_ID'] = x1[:N1]
test['new_ID'] = x1[N1:]

# x2 = np.concatenate((x12,x22))
# x2 = preprocessing.LabelEncoder().fit_transform(x2) #13963746x21567
# print(x2.shape)#(14454126, 21567)
# train['new_e_ID'] = x2[:N1]
# test['new_e_ID'] = x2[N1:]

# x3 = np.concatenate((x13,x23))
# x3 = preprocessing.LabelEncoder().fit_transform(x3) #13963746x21567
# print(x3.shape)#(14454126, 21567)
# train['new_s_ID'] = x3[:N1]
# test['new_s_ID'] = x3[N1:]


/home/b418-xiwei/anaconda3/envs/dl/lib/python3.6/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(14223933,)


In [2]:
# np.unique(x3).shape

# from sklearn.utils import shuffle
# line = shuffle(train)
# train_num = int(0.9 * line.shape[0])
# train = line[:train_num]
# dev = line[train_num:]
# print(train.shape)
# print(dev.shape)

(12360197, 38)
(1373356, 38)


In [2]:
from keras.layers import Reshape, Conv2D, MaxPooling2D
def get_model():
#     col1 = ['new_ID']
#     # col2 = ['new_e_ID']
#     # col3 = ['new_s_ID']
#     col4 = [c for c in train if
#            c not in ['new_e_ID','new_s_ID', 'is_rain', 'hour_s', 'hour_e', 'ID2', 'new_ID2','Unnamed: 0', 'is_crowd', 's_ij', 'e_ij','s_x', 's_y', 'e_x', 'e_y','new_ID','ID','O_LINENO', 'O_UP', 'Source_Station', 'Target_Station', 'O_TIME', 'aver_v', 'max_v',
#                      'Diff_Time']]
#     #+gps 2 epoch 31
#     X_train1 = train[col1].values
#     # X_train2 = train[col2].values
#     # X_train3 = train[col3].values
#     X_train4 = train[col4].values

#     y_train = train['Diff_Time'].values
#     print(col1,col4)#col2,col3,
#     print(X_train1.shape)
#     X_dev1 = dev[col1].values
#     # X_dev2 = dev[col2].values
#     # X_dev3 = dev[col3].values
#     X_dev4 = dev[col4].values

#     y_dev = dev['Diff_Time'].values
    # print(X_dev.shape, y_dev.shape)

    input1 = Input(shape=(1,))
    # input2 = Input(shape=(1,))
    # input3 = Input(shape=(1,))
    input4 = Input(shape=(16,))

    x1 = Embedding(21449, 256, input_length=1)(input1)
    x1 = Flatten()(x1)
    # x2 = Embedding(425, 128, input_length=1)(input2)
    # x2 = Flatten()(x2)
    # x3 = Embedding(434, 128, input_length=1)(input3)
    # x3 = Flatten()(x3)
    x4 = Dense(units=256)(input4)
    x4 = Activation("relu")(x4)

    x = concatenate([x1,x4])#,x2,x3
    #CNN
    # x = Reshape((20, 20, 1))(x)
    # x = Conv2D(32, (3, 3), padding='same')(x)
    # x = MaxPooling2D(pool_size=(2, 2))(x)
    # x = Conv2D(64, (3, 3), padding='same')(x)
    # x = MaxPooling2D(pool_size=(2, 2))(x)
    # x = Flatten()(x)
    x = Dense(2048, activation='relu')(x)
#     x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    out = Dense(1)(x)
    
    model = Model(inputs=[input1,input4], outputs=out)#,input2,input3
# print(model.summary())
# # model.load_weights('./log/weights-21.7671.hdf5')
    model.compile(loss=root_mean_squared_error, optimizer='adam', metrics=[root_mean_squared_error])
# model.fit([X_train1,X_train4], y_train, validation_data=([X_dev1,X_dev4], y_dev), epochs=40, batch_size=1024,callbacks=[ckpt,earlyStopping])
# model.save("keras3.h5"),X_train2,X_train3 ,X_dev2,X_dev3
    return model


In [14]:
col1 = ['new_ID']
# col2 = ['new_e_ID']
# col3 = ['new_s_ID']

col4 = [c for c in test if
       c not in [ 'new_e_ID', 'is_rain', 'new_s_ID','hour_e','hour_s','is_crowd', 's_ij', 'e_ij','s_x', 's_y', 'e_x', 'e_y','ID2', 'new_ID2','Unnamed: 0','new_ID','ID','O_LINENO', 'O_UP', 'Source_Station', 'Target_Station', 'O_TIME', 'aver_v', 'max_v',
                 'Diff_Time','Distance1', 'distance2','TERMINALNO', 'new_dist']]
print(col1,col4)#,col2,col3
X_test1 = test[col1].values
# X_test2 = test[col2].values
# X_test3 = test[col3].values
X_test4 = test[col4].values

print(X_test1.shape,X_test4.shape)#,X_test2.shape,X_test3.shape

# model.load_weights('./log/weights-23.5405.hdf5') #23.4278 0.0366
# y_pred =  model.predict([X_test1,X_test4],batch_size=1024)[:,0]#X_test2,X_test3


['new_ID'] ['Distance', 'hour', 'is_peek', 'weekday', 'is_workday', 'dws', 'nws', 'dts', 'nts', 'h_aver_diff', 'h_aver_d', 'h_aver_v', 'grid_aver_diff', 'grid_aver_d', 'hour_weekday', 'hour_crowd']
(490380, 1) (490380, 16)


In [15]:
test = test[['ID', 'new_dist',  'O_LINENO', 'O_UP', 'Source_Station', 'Target_Station',
       'Distance', 'distance2', 'O_TIME', 'hour', 'is_peek', 'weekday',
       'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'max_v',
       'h_aver_diff', 'h_aver_d', 'h_aver_v', 'TERMINALNO', 
       'hour_weekday','is_crowd', 's_ij', 'e_ij']]
test.columns=['ID', 'Distance',  'O_LINENO', 'O_UP', 'Source_Station', 'Target_Station',
       'Distance1', 'distance2', 'O_TIME', 'hour', 'is_peek', 'weekday',
       'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'max_v',
       'h_aver_diff', 'h_aver_d', 'h_aver_v', 'TERMINALNO', 
       'hour_weekday','is_crowd', 's_ij', 'e_ij']

test['pred1'] = pred
test['pred2'] = pred

sub1 = test[['O_LINENO','TERMINALNO', 'O_UP','Source_Station','Target_Station','O_TIME','pred1','pred2','Distance','Distance1']]
sub1.columns = ['LINE','TERMINALNO','UP','pred_start_stop_ID','pred_end_stop_ID','realTime','pred1','pred2','Distance','Distance1']
sub1 = sub1.reset_index()
del sub1['index']
sub1


,LINE,TERMINALNO,UP,pred_start_stop_ID,pred_end_stop_ID,realTime,pred1,pred2,Distance,Distance1
0,866,905578,0,8,9,2017-10-25 09:00:00,108.601122,108.601122,0.359933,0.629479
1,866,905578,0,9,10,2017-10-25 09:00:00,127.710429,127.710429,0.517842,0.517842
2,866,905578,0,10,11,2017-10-25 09:00:00,131.744756,131.744756,0.536539,0.536539
3,866,905578,0,11,12,2017-10-25 09:00:00,179.018715,179.018715,0.776565,0.776565
4,866,905578,0,12,13,2017-10-25 09:00:00,64.819788,64.819788,0.408048,0.408048
5,866,905578,0,13,14,2017-10-25 09:00:00,151.118077,151.118077,0.983655,0.983655
6,866,905578,0,14,15,2017-10-25 09:00:00,120.443813,120.443813,0.586326,0.586326
7,866,905578,0,15,16,2017-10-25 09:00:00,70.933391,70.933391,0.480220,0.480220
8,866,905578,0,16,17,2017-10-25 09:00:00,201.469424,201.469424,0.520341,0.520341
9,866,905578,0,17,18,2017-10-25 09:00:00,218.961398,218.961398,0.794807,0.794807


In [16]:
sub=pd.read_csv("./toBePredicted_0607_segment.csv", sep=",")
# sub['realTime'] = pd.to_datetime(sub['realTime'],format='%Y-%m-%d %H:%M:%S')
sub2 = sub[['LINE','TERMINALNO','UP','pred_start_stop_ID','pred_end_stop_ID','realTime','distance']]
sub2=pd.merge(sub2,sub1,on=['LINE','TERMINALNO','UP','pred_start_stop_ID','pred_end_stop_ID','realTime'],how='left')
sub2

,LINE,TERMINALNO,UP,pred_start_stop_ID,pred_end_stop_ID,realTime,distance,pred1,pred2,Distance,Distance1
0,678,904388,0,11,12,2017-10-28 14:00:00,0.417984,129.199532,129.199532,0.679068,1.097053
1,678,904388,0,12,13,2017-10-28 14:00:00,0.000000,152.586206,152.586206,0.744740,0.744740
2,678,904388,0,13,14,2017-10-28 14:00:00,0.000000,68.265110,68.265110,0.479191,0.479191
3,678,904388,0,14,15,2017-10-28 14:00:00,0.000000,91.065974,91.065974,0.367919,0.367919
4,678,904388,0,15,16,2017-10-28 14:00:00,0.000000,249.676459,249.676459,1.141722,1.141722
5,678,904388,0,16,17,2017-10-28 14:00:00,0.000000,139.200336,139.200336,0.578620,0.578620
6,678,904388,0,17,18,2017-10-28 14:00:00,0.000000,74.664507,74.664507,0.410412,0.410412
7,678,904388,0,18,19,2017-10-28 14:00:00,0.000000,119.506632,119.506632,0.665032,0.665032
8,678,904388,0,19,20,2017-10-28 14:00:00,0.000000,182.003813,182.003813,0.907266,0.907266
9,678,904388,0,20,21,2017-10-28 14:00:00,0.000000,145.500189,145.500189,0.471062,0.471062


In [17]:
sub2['div_dist'] = sub2['Distance'] / sub2['Distance1']
sub2['new_pred'] = sub2['div_dist'] * sub2['pred1']

import math
for i in range(sub2.shape[0]):
    s = sub2.iloc[i]
    if math.isnan(s['div_dist']):
        sub2.loc[i,'new_pred'] = s['pred1']
        
#计算用newpred和直接用新距离之间的偏差
print((sub2['pred1']-sub2['pred2']).mean())
print((sub2['pred2']-sub2['new_pred']).mean())
print((sub2['pred1']-sub2['new_pred']).mean())

0.0
4.3713018216540185
4.3713018216540185


In [18]:
sub2['new_pred'].mean()


128.38579903195637

In [19]:
sub2.to_csv('./toBePredicted_0702_resultn.csv',sep=",",index=False)#0605 29.2467

print('finished')

finished


In [6]:
def train2(train,test,n_folds,epoch=40):
    from sklearn.model_selection import StratifiedKFold, KFold
    print("folding")
    y = train['Diff_Time'].values
    print(y)
    kf = KFold(n_splits=n_folds,shuffle=True)
    result = np.zeros((len(test), 1))

    col1 = ['new_ID']
    # col2 = ['new_e_ID']
    # col3 = ['new_s_ID']
    col4 = [c for c in test if
           c not in [ 'new_e_ID', 'is_rain', 'new_s_ID','hour_e','hour_s','is_crowd', 's_ij', 'e_ij','s_x', 's_y', 'e_x', 'e_y','ID2', 'new_ID2','Unnamed: 0','new_ID','ID','O_LINENO', 'O_UP', 'Source_Station', 'Target_Station', 'O_TIME', 'aver_v', 'max_v',
                     'Diff_Time','Distance1', 'distance2','TERMINALNO', 'new_dist']]
    print(col1,col4)#,col2,col3
    X_test1 = test[col1].values
    # X_test2 = test[col2].values
    # X_test3 = test[col3].values
    X_test4 = test[col4].values

    print('test',X_test1.shape,X_test4.shape)

    print("training")
    count = 0
    for (tr_idx, val_idx) in kf.split(y):
#         print(tr_idx, val_idx)
        col1 = ['new_ID']
        col4 = [c for c in train if
               c not in ['new_e_ID','new_s_ID', 'is_rain', 'hour_s', 'hour_e', 'ID2', 'new_ID2','Unnamed: 0', 'is_crowd', 's_ij', 'e_ij','s_x', 's_y', 'e_x', 'e_y','new_ID','ID','O_LINENO', 'O_UP', 'Source_Station', 'Target_Station', 'O_TIME', 'aver_v', 'max_v',
                     'Diff_Time']]
        X_train1 = train[col1].values[tr_idx]
        X_train4 = train[col4].values[tr_idx]
        y_train = y[tr_idx]

        y_dev = y[val_idx]
        X_dev1 = train[col1].values[val_idx]
        X_dev4 = train[col4].values[val_idx]

        print(X_train1.shape,X_train4.shape)
        print(X_dev1.shape,X_dev4.shape)

        model = get_model()
        if count == 0:
            print(model.summary())
        print(count)
        model.fit([X_train1, X_train4], y_train, validation_data=([X_dev1, X_dev4], y_dev), epochs=40, batch_size=1024,
                  callbacks=[earlyStopping])

        result += model.predict([X_test1, X_test4], batch_size=1024)
        y_te_pred = model.predict([X_dev1, X_dev4], batch_size=1024)
        #print(log_loss(y_dev, y_te_pred))
        count = count+1
        # 释放显存
        K.clear_session()
        tf.reset_default_graph()
    # 提交结果
    result /= n_folds
    return result

In [3]:
print(train.shape,test.shape)

(13733553, 38) (490380, 39)


In [8]:
import tensorflow as tf
pred = train2(train,test,n_folds=10,epoch=40)

folding
[140 230  70 ... 150 317 130]
['new_ID'] ['Distance', 'hour', 'is_peek', 'weekday', 'is_workday', 'dws', 'nws', 'dts', 'nts', 'h_aver_diff', 'h_aver_d', 'h_aver_v', 'grid_aver_diff', 'grid_aver_d', 'hour_weekday', 'hour_crowd']
test (490380, 1) (490380, 16)
training
(12360197, 1) (12360197, 16)
(1373356, 1) (1373356, 16)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 16)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 256)       5490944     input_1[0]

Train on 12360197 samples, validate on 1373356 samples
Epoch 1/40
12360197/12360197 [==============================] - 247s 20us/step - loss: 26.8271 - root_mean_squared_error: 26.8271 - val_loss: 24.1539 - val_root_mean_squared_error: 24.1539
Epoch 2/40
12360197/12360197 [==============================] - 246s 20us/step - loss: 25.4217 - root_mean_squared_error: 25.4217 - val_loss: 23.9482 - val_root_mean_squared_error: 23.9482
Epoch 3/40
12360197/12360197 [==============================] - 246s 20us/step - loss: 24.9681 - root_mean_squared_error: 24.9681 - val_loss: 23.9689 - val_root_mean_squared_error: 23.9689
Epoch 4/40
12360197/12360197 [==============================] - 246s 20us/step - loss: 24.6660 - root_mean_squared_error: 24.6660 - val_loss: 23.7687 - val_root_mean_squared_error: 23.7687
Epoch 5/40
12360197/12360197 [==============================] - 247s 20us/step - loss: 24.4440 - root_mean_squared_error: 24.4440 - val_loss: 23.5635 - val_root_mean_squared_error: 23.5635


Epoch 22/40
12360198/12360198 [==============================] - 244s 20us/step - loss: 22.8981 - root_mean_squared_error: 22.8981 - val_loss: 23.3234 - val_root_mean_squared_error: 23.3234
Epoch 23/40
12360198/12360198 [==============================] - 244s 20us/step - loss: 22.8456 - root_mean_squared_error: 22.8456 - val_loss: 23.3596 - val_root_mean_squared_error: 23.3596
(12360198, 1) (12360198, 16)
(1373355, 1) (1373355, 16)
5
Train on 12360198 samples, validate on 1373355 samples
Epoch 1/40
12360198/12360198 [==============================] - 245s 20us/step - loss: 26.8441 - root_mean_squared_error: 26.8441 - val_loss: 24.3031 - val_root_mean_squared_error: 24.3031
Epoch 2/40
12360198/12360198 [==============================] - 244s 20us/step - loss: 25.3850 - root_mean_squared_error: 25.3850 - val_loss: 24.2191 - val_root_mean_squared_error: 24.2191
Epoch 3/40
12360198/12360198 [==============================] - 245s 20us/step - loss: 24.9985 - root_mean_squared_error: 24.9985

Epoch 3/40
12360198/12360198 [==============================] - 248s 20us/step - loss: 25.0271 - root_mean_squared_error: 25.0271 - val_loss: 24.0534 - val_root_mean_squared_error: 24.0534
Epoch 4/40
12360198/12360198 [==============================] - 247s 20us/step - loss: 24.7386 - root_mean_squared_error: 24.7386 - val_loss: 24.0099 - val_root_mean_squared_error: 24.0099
Epoch 5/40
12360198/12360198 [==============================] - 247s 20us/step - loss: 24.5281 - root_mean_squared_error: 24.5281 - val_loss: 23.6266 - val_root_mean_squared_error: 23.6266
Epoch 6/40
12360198/12360198 [==============================] - 247s 20us/step - loss: 24.3436 - root_mean_squared_error: 24.3436 - val_loss: 23.6928 - val_root_mean_squared_error: 23.6928
Epoch 7/40
12360198/12360198 [==============================] - 246s 20us/step - loss: 24.1831 - root_mean_squared_error: 24.1831 - val_loss: 23.5848 - val_root_mean_squared_error: 23.5848
Epoch 8/40
12360198/12360198 [=========================